# Problem Formulation

## Problem Description:
This project aims to develop a machine learning model that predicts future sales
and demand by utilising historical sales data and external factors, including
product details, promotions, seasonality, holidays, and economic indicators. The
goal is to analyse historical patterns and generate reliable forecasts that help
businesses make data-driven decisions to reduce costs, increase efficiency, and
improve customer satisfaction by predicting the daily sales for the next 28 days.

## Objectives

● Collect and preprocess historical sales and demand data.

● Identify key features that influence sales trends.

● Build, train, and optimise forecasting models to predict future sales and
demand.

● Deploy the best-performing model to generate forecasts in real-time or in
batches.
## Data source:
Hierarchical sales data from Walmart, the world’s largest company by revenue in the US.



# Code setup

## Important libraries

In [1]:
import pandas as pd
import numpy as np

## data reading

In [2]:
#sales_validation=pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')
#sales_validation.head(3)

In [3]:
#cal=pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
#cal.head(3)

In [4]:
#sell_price = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sell_prices.csv")
#sell_price.head(3)

In [5]:
data = pd.read_csv("/kaggle/input/depi-dataset/data.csv")

# Schema formating

## sales_validation file

In [6]:
'''
data = sales_validation.melt(
    id_vars=["id","item_id", "dept_id","cat_id","store_id","state_id"],  # columns to keep
    var_name="d",                          # new column name for day labels (d_1, d_2, ...)
    value_name="sales"                     # new column name for sales values
)
'''


'\ndata = sales_validation.melt(\n    id_vars=["id","item_id", "dept_id","cat_id","store_id","state_id"],  # columns to keep\n    var_name="d",                          # new column name for day labels (d_1, d_2, ...)\n    value_name="sales"                     # new column name for sales values\n)\n'

In [7]:
#data.drop(columns=['id'],inplace=True)

## calendar file

In [8]:
#cal["date"]= pd.to_datetime(cal["date"])

In [9]:
'''
cal["event_name_1"]= cal["event_name_1"].fillna("No event")
cal["event_type_1"]= cal["event_type_1"].fillna("No event")
cal["event_name_2"]= cal["event_name_2"].fillna("No event")
cal["event_type_2"]= cal["event_type_2"].fillna("No event")
'''

'\ncal["event_name_1"]= cal["event_name_1"].fillna("No event")\ncal["event_type_1"]= cal["event_type_1"].fillna("No event")\ncal["event_name_2"]= cal["event_name_2"].fillna("No event")\ncal["event_type_2"]= cal["event_type_2"].fillna("No event")\n'

In [10]:
'''
# Merge data and cal dataframes on the 'd' column
merged_data = pd.merge(data, cal, on='d', how='left')

conditions = [
    merged_data["state_id"] == "CA",
    merged_data["state_id"] == "TX",
    merged_data["state_id"] == "WI"
]
choices= [
    merged_data["snap_CA"],
    merged_data["snap_TX"],
    merged_data["snap_WI"]
]
merged_data["snap"]= np.select(conditions, choices)

merged_data.drop(columns=['snap_CA','snap_TX','snap_WI'],inplace=True)
'''

'\n# Merge data and cal dataframes on the \'d\' column\nmerged_data = pd.merge(data, cal, on=\'d\', how=\'left\')\n\nconditions = [\n    merged_data["state_id"] == "CA",\n    merged_data["state_id"] == "TX",\n    merged_data["state_id"] == "WI"\n]\nchoices= [\n    merged_data["snap_CA"],\n    merged_data["snap_TX"],\n    merged_data["snap_WI"]\n]\nmerged_data["snap"]= np.select(conditions, choices)\n\nmerged_data.drop(columns=[\'snap_CA\',\'snap_TX\',\'snap_WI\'],inplace=True)\n'

## sell_price file


In [11]:
'''
final_data = temp_data.merge(
        sell_price,
        on=["store_id", "item_id", "wm_yr_wk"],
        how="left",
        validate="m:1"  # many sales rows per unique price row is expected
    )
    '''

'\nfinal_data = temp_data.merge(\n        sell_price,\n        on=["store_id", "item_id", "wm_yr_wk"],\n        how="left",\n        validate="m:1"  # many sales rows per unique price row is expected\n    )\n    '

In [12]:
#final_data["price_in_dollars"]=final_data["sell_price"]*final_data["sales"]

In [13]:
#final_data.to_csv("/kaggle/working/data.csv")

# EDA

In [14]:
data.loc[data["sell_price"].isna(), "sales"].value_counts()

sales
0    12299413
Name: count, dtype: int64

In [15]:
data.isna().sum()

Unnamed: 0                 0
item_id                    0
dept_id                    0
cat_id                     0
store_id                   0
state_id                   0
d                          0
sales                      0
date                       0
wm_yr_wk                   0
weekday                    0
wday                       0
month                      0
year                       0
event_name_1               0
event_type_1               0
event_name_2               0
event_type_2               0
snap                       0
sell_price          12299413
price_in_dollars    12299413
dtype: int64

In [16]:
# Choose numeric columns to check
numeric_cols = ["sales", "sell_price", "price_in_dollars"]

# Create a dictionary to store results
outlier_summary = {}

for col in numeric_cols:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1

    # Define outlier thresholds
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    # Boolean mask for outliers
    mask = (data[col] < lower) | (data[col] > upper)

    # Store summary
    outlier_summary[col] = {
        "Q1": Q1,
        "Q3": Q3,
        "Lower Bound": lower,
        "Upper Bound": upper,
        "Outlier Count": mask.sum(),
        "Outlier %": round(mask.mean() * 100, 3)
    }

# Convert to DataFrame for readability
outlier_df = pd.DataFrame(outlier_summary).T
print("🔍 Outlier Summary:")
display(outlier_df)


🔍 Outlier Summary:


,Q1,Q3,Lower Bound,Upper Bound,Outlier Count,Outlier %
sales,0.00,1.00,-1.50,2.50,6902589.0,11.834
sell_price,2.18,5.84,-3.31,11.33,2035260.0,3.489
price_in_dollars,0.00,4.78,-7.17,11.95,4249172.0,7.285


In [17]:
Q1 = data["price_in_dollars"].quantile(0.25)
Q3 = data["price_in_dollars"].quantile(0.75)
IQR = Q3 - Q1
upper = Q3 + 1.5 * IQR

data[data["price_in_dollars"] > upper].sort_values("price_in_dollars", ascending=False).head(10)


,Unnamed: 0,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap,sell_price,price_in_dollars
15552906,15552906,FOODS_3_785,FOODS_3,FOODS,CA_1,CA,d_511,648,2012-06-22,11221,...,7,6,2012,No event,No event,No event,No event,0,3.34,2164.32
26173774,26173774,HOUSEHOLD_2_062,HOUSEHOLD_2,HOUSEHOLD,TX_1,TX,d_859,601,2013-06-05,11319,...,5,6,2013,No event,No event,No event,No event,1,2.94,1766.94
44229139,44229139,HOBBIES_1_354,HOBBIES_1,HOBBIES,TX_3,TX,d_1451,81,2015-01-18,11451,...,2,1,2015,No event,No event,No event,No event,0,19.98,1618.38
48040389,48040389,HOBBIES_1_354,HOBBIES_1,HOBBIES,TX_3,TX,d_1576,69,2015-05-23,11517,...,1,5,2015,No event,No event,No event,No event,0,22.98,1585.62
2971837,2971837,FOODS_2_285,FOODS_2,FOODS,TX_1,TX,d_98,634,2011-05-06,11114,...,7,5,2011,No event,No event,No event,No event,1,2.36,1496.24
17717696,17717696,FOODS_3_785,FOODS_3,FOODS,CA_1,CA,d_582,427,2012-09-01,11232,...,1,9,2012,No event,No event,No event,No event,1,3.34,1426.18
18668322,18668322,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,d_613,276,2012-10-02,11236,...,4,10,2012,No event,No event,No event,No event,1,4.98,1374.48
45296097,45296097,HOBBIES_1_158,HOBBIES_1,HOBBIES,TX_3,TX,d_1486,56,2015-02-22,11504,...,2,2,2015,No event,No event,No event,No event,0,22.98,1286.88
18698812,18698812,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,d_614,256,2012-10-03,11236,...,5,10,2012,No event,No event,No event,No event,1,4.98,1274.88
45041616,45041616,FOODS_1_180,FOODS_1,FOODS,CA_3,CA,d_1478,130,2015-02-14,11503,...,1,2,2015,ValentinesDay,Cultural,No event,No event,0,9.67,1257.10


In [18]:
# Calculate IQR-based upper threshold
Q1 = data["price_in_dollars"].quantile(0.25)
Q3 = data["price_in_dollars"].quantile(0.75)
IQR = Q3 - Q1
upper = Q3 + 1.5 * IQR

# Extract outliers (above upper bound)
outliers = data[data["price_in_dollars"] > upper]

# Show a random sample of 10 outlier rows
outliers.sample(10, random_state=42)


,Unnamed: 0,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap,sell_price,price_in_dollars
21960941,21960941,FOODS_2_217,FOODS_2,FOODS,CA_3,CA,d_721,4,2013-01-18,11251,...,7,1,2013,No event,No event,No event,No event,0,3.98,15.92
16658687,16658687,FOODS_2_174,FOODS_2,FOODS,CA_4,CA,d_547,6,2012-07-28,11227,...,1,7,2012,No event,No event,No event,No event,0,2.82,16.92
21811614,21811614,FOODS_2_291,FOODS_2,FOODS,CA_4,CA,d_716,2,2013-01-13,11251,...,2,1,2013,No event,No event,No event,No event,0,6.98,13.96
6851124,6851124,HOBBIES_1_022,HOBBIES_1,HOBBIES,WI_1,WI,d_225,2,2011-09-10,11133,...,1,9,2011,No event,No event,No event,No event,0,6.98,13.96
15584350,15584350,HOUSEHOLD_1_354,HOUSEHOLD_1,HOUSEHOLD,CA_2,CA,d_512,7,2012-06-23,11222,...,1,6,2012,No event,No event,No event,No event,0,1.93,13.51
28340023,28340023,FOODS_3_393,FOODS_3,FOODS,TX_1,TX,d_930,4,2013-08-15,11329,...,6,8,2013,No event,No event,No event,No event,1,3.50,14.00
47713437,47713437,FOODS_3_461,FOODS_3,FOODS,WI_2,WI,d_1565,11,2015-05-12,11515,...,4,5,2015,No event,No event,No event,No event,1,1.48,16.28
54567205,54567205,FOODS_3_077,FOODS_3,FOODS,TX_3,TX,d_1790,11,2015-12-23,11547,...,5,12,2015,No event,No event,No event,No event,0,2.50,27.50
8841867,8841867,FOODS_3_592,FOODS_3,FOODS,WI_3,WI,d_290,2,2011-11-14,11142,...,3,11,2011,No event,No event,No event,No event,1,6.98,13.96
26798334,26798334,HOUSEHOLD_1_113,HOUSEHOLD_1,HOUSEHOLD,WI_3,WI,d_879,5,2013-06-25,11322,...,4,6,2013,No event,No event,No event,No event,0,6.47,32.35


In [19]:
print(data["event_name_1"].unique())
print(data["event_name_2"].unique())
print(data["event_type_1"].unique())
print(data["event_type_2"].unique())

['No event' 'SuperBowl' 'ValentinesDay' 'PresidentsDay' 'LentStart'
 'LentWeek2' 'StPatricksDay' 'Purim End' 'OrthodoxEaster' 'Pesach End'
 'Cinco De Mayo' "Mother's day" 'MemorialDay' 'NBAFinalsStart'
 'NBAFinalsEnd' "Father's day" 'IndependenceDay' 'Ramadan starts'
 'Eid al-Fitr' 'LaborDay' 'ColumbusDay' 'Halloween' 'EidAlAdha'
 'VeteransDay' 'Thanksgiving' 'Christmas' 'Chanukah End' 'NewYear'
 'OrthodoxChristmas' 'MartinLutherKingDay' 'Easter']


['No event' 'Easter' 'Cinco De Mayo' 'OrthodoxEaster' "Father's day"]


['No event' 'Sporting' 'Cultural' 'National' 'Religious']


['No event' 'Cultural' 'Religious']


In [20]:
data.head(2000)

,Unnamed: 0,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap,sell_price,price_in_dollars
0,0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,No event,No event,No event,No event,0,NaN,NaN
1,1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,No event,No event,No event,No event,0,NaN,NaN
2,2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,No event,No event,No event,No event,0,NaN,NaN
3,3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,No event,No event,No event,No event,0,NaN,NaN
4,4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,No event,No event,No event,No event,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,FOODS_2_169,FOODS_2,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,No event,No event,No event,No event,0,NaN,NaN
1996,1996,FOODS_2_170,FOODS_2,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,No event,No event,No event,No event,0,NaN,NaN
1997,1997,FOODS_2_171,FOODS_2,FOODS,CA_1,CA,d_1,4,2011-01-29,11101,...,1,1,2011,No event,No event,No event,No event,0,2.98,11.92
1998,1998,FOODS_2_172,FOODS_2,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,No event,No event,No event,No event,0,NaN,NaN


# Data-Cleaning

In [21]:
data.drop(columns = "sell_price",inplace = True)
data.drop(columns="Unnamed: 0",inplace = True)
data.drop(columns="dept_id",inplace = True)
data.drop(columns = "cat_id",inplace = True)
data.drop(columns = "state_id",inplace = True)
data.drop(columns="month",inplace = True)
data.drop(columns="year",inplace = True)
data.drop(columns="weekday",inplace=True)

In [22]:
data.fillna(0,inplace = True)

In [23]:
data.sort_values(by=["store_id", "item_id", "d"], inplace=True)
data.reset_index(drop=True, inplace=True)

In [24]:
data.to_csv("/kaggle/working/data.csv")